In [ ]:
import numpy as np
import cv2

def filtro_media(imagem, janela_ref, diferenca_limite):
    count = 0
    
    imagem_filtrada = imagem.copy()

    for i in range(5, imagem.shape[0] - 5):
        for j in range(5, imagem.shape[1] - 5):
            janela_prin = imagem[i-3:i+4, j-3:j+4]
            
            def correlacao_ncc(janela_prin, janela_ref):
                
                if np.std(janela_prin) == 0:
                    janela_prin_norm = 0
                else:
                    janela_prin_norm = (janela_prin - np.mean(janela_prin)) / np.std(janela_prin)
                    
                janela_ref_norm = (janela_ref - np.mean(janela_ref)) / np.std(janela_ref)
                
                denominador = np.sqrt(np.sum(janela_prin_norm**2) * np.sum(janela_ref_norm**2))
                
                if denominador == 0:
                    ncc = 0
                else:
                    ncc = np.sum(janela_prin_norm * janela_ref_norm) / denominador
                return ncc

            ncc = correlacao_ncc(janela_prin, janela_ref)

            if ncc > diferenca_limite:
                count += 1

                for a in range(-2, 3):
                    for b in range(-2, 3):
                        window2 = imagem[i-3+a:i+3+1+a, j-3+b:j+3+1+b]
                        imagem_filtrada[i+a,j+b] = np.median(window2)

    print('Artefatos removidos:',count)
    return imagem_filtrada

imagem = cv2.imread('exemplo.png', cv2.IMREAD_GRAYSCALE)

janela_ref = np.array([[251, 252, 250, 255, 255, 251, 249],
                       [246, 240, 233, 245, 248, 252, 253],
                       [245, 224, 204, 203, 219, 239, 252],
                       [245, 212, 180, 161, 188, 223, 248],
                       [250, 232, 216, 197, 215, 237, 249],
                       [255, 245, 237, 227, 238, 250, 255],
                       [255, 254, 253, 249, 251, 253, 253]])

diferenca_limite = 0.4

imagem_processada = filtro_media(imagem, janela_ref, diferenca_limite)

cv2.imwrite('imagem_processada.png', imagem_processada)